In [1]:
# scripts/compare_pliego_vs_propuestas.py
import json, glob, os
from difflib import SequenceMatcher

def sim(a,b):
    return SequenceMatcher(None, a or "", b or "").ratio()

def score_proposal(pliego, prop):
    score = 0
    reasons = []
    # existencia de secciones
    if prop.get("requisitos_tecnicos"): score += 35
    else: reasons.append("Falta requisitos técnicos")
    ce = prop.get("condiciones_economicas","")
    if ce:
        # si incluye monto / número -> mejor puntaje
        if any(ch.isdigit() for ch in ce): score += 30
        else: score += 10; reasons.append("Condiciones económicas no numéricas")
    else:
        reasons.append("Falta condiciones económicas")
    if prop.get("clausulas_legales"): score += 35
    else: reasons.append("Falta cláusulas legales")
    # comparación textual con el pliego: similitud en requisitos_tecnicos
    sim_tecn = sim(pliego.get("requisitos_tecnicos",""), prop.get("requisitos_tecnicos",""))
    score += int(sim_tecn * 20)  # penaliza/beneficia por similitud
    score = min(score, 100)
    return score, reasons, {"sim_tecn": sim_tecn}

def semaforo(score):
    return "VERDE" if score>=75 else ("AMARILLO" if score>=50 else "ROJO")

def main(pliego_path="processed/extracted_info.json", proposals_dir="processed/propuestas_estructuradas", out="processed/summary_comparison.json"):
    with open(pliego_path, "r", encoding="utf-8") as f:
        pliego = json.load(f)
    results = {}
    for p in glob.glob(os.path.join(proposals_dir,"*.json")):
        with open(p,"r",encoding="utf-8") as f:
            prop = json.load(f)
        score, reasons, meta = score_proposal(pliego, prop)
        results[os.path.basename(p)] = {
            "score": score,
            "semaforo": semaforo(score),
            "reasons": reasons,
            "meta": meta
        }
    os.makedirs(os.path.dirname(out), exist_ok=True)
    with open(out,"w",encoding="utf-8") as f:
        json.dump(results,f,indent=2,ensure_ascii=False)
    print("Resumen comparativo guardado en", out)

if __name__ == "__main__":
    main()


Resumen comparativo guardado en processed/summary_comparison.json
